In [1]:
import os
import TrainData_Unet
import torch
import matplotlib.pyplot as plt
import models.UNet as model
from torch.utils.data import DataLoader
import numpy as np
import torch.optim as optim
import time
import torch.nn as nn

In [2]:
gpu_id = 1
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

device = torch.device('cuda')

In [3]:
unet = model.UNet(1,256,64)
unet_params = list(unet.parameters())
nb_param=0
for param in unet.parameters():
    nb_param+=np.prod(list(param.data.size()))
print(nb_param)
unet = unet.to(device)

31058944


In [ ]:
losses = []
for iteration in range(10):
    start=time.time()
    for sub_idx in range(330):

        T1a_dir = '/home/xiaoyu/MRIdata/T1w/axial/sub{}'.format(sub_idx)
   
        parc5a_dir = '/home/xiaoyu/MRIdata/parc_5/axial/sub{}'.format(sub_idx)
   
        T1a_list = os.listdir(T1a_dir)
  
        parc5a_list = os.listdir(parc5a_dir)

    
        if sub_idx == 0: # set sub0 as test set.
            print('\nT1w Axial slices num:',len(T1a_list))
            print('\nParc5 Axial slices num:',len(parc5a_list))

            continue
        
        print('\nSubject num:',sub_idx)   
        train_data = TrainData_Unet.TrainDataset(T1a_dir=T1a_dir, parc5a_dir = parc5a_dir)
        dataloader = DataLoader(train_data, batch_size = 5, shuffle = True, num_workers = 4)
    
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(unet.parameters() ,lr=0.001)
    
        for epoch in range(0,3):
   
    
            # define the running loss
            running_loss = 0
            running_error = 0
            num_batches=0
      
            for i_batch, sample_batched in enumerate(dataloader):
        
                optimizer.zero_grad()
        
                #get the inputs
                inputs, labels = sample_batched['T1a'], sample_batched['parc5a']
        
                inputs = inputs.to(device)
                labels = labels.to(device)

                inputs.requires_grad_()
        
                #forward + backward +optimize
                scores = unet(inputs)
          
                # Define the loss
                loss = criterion(scores, labels.long()) 
                loss.backward()
                optimizer.step()
        
                # compute and accumulate stats
                running_loss += loss.detach().item()
       
                num_batches+=1 
    
            # AVERAGE STATS THEN DISPLAY    
            total_loss = running_loss/num_batches
   
            elapsed = (time.time()-start)/60
                 
            print('epoch=',epoch, '\t time=', elapsed,'min', '\t loss=', total_loss )
            losses.append(total_loss)
       
        print('Finish Training for one subject')
        torch.save(losses,'loss'+str(sub_idx)+'.pt')
    print(iteration,'Iteration')


T1w Axial slices num: 182

Parc5 Axial slices num: 182

Subject num: 1
epoch= 0 	 time= 0.07145477135976155 min 	 loss= 4.799524887188061
epoch= 1 	 time= 0.14550222158432008 min 	 loss= 3.6760572678334005
epoch= 2 	 time= 0.2182711402575175 min 	 loss= 2.2482401390333435
Finish Training for one subject

Subject num: 2
epoch= 0 	 time= 0.28983285029729206 min 	 loss= 1.2777650581823814
epoch= 1 	 time= 0.36485706170399984 min 	 loss= 0.9321625168259079
epoch= 2 	 time= 0.4353734016418457 min 	 loss= 0.8766698837280273
Finish Training for one subject

Subject num: 3
epoch= 0 	 time= 0.5100964665412903 min 	 loss= 0.7864431843564317
epoch= 1 	 time= 0.5804174621899922 min 	 loss= 0.7565836302331976
epoch= 2 	 time= 0.6543830792109172 min 	 loss= 0.7484367751591915
Finish Training for one subject

Subject num: 4
epoch= 0 	 time= 0.7258615891138712 min 	 loss= 0.8617854529135937
epoch= 1 	 time= 0.7999736944834391 min 	 loss= 0.8376884770554465
epoch= 2 	 time= 0.8701321164766948 min 	 lo

epoch= 0 	 time= 7.309643030166626 min 	 loss= 0.7011080271488911
epoch= 1 	 time= 7.384240587552388 min 	 loss= 0.6655900784440942
epoch= 2 	 time= 7.4558084805806475 min 	 loss= 0.6559825291907465
Finish Training for one subject

Subject num: 35
epoch= 0 	 time= 7.5315491159756975 min 	 loss= 0.7080720406327699
epoch= 1 	 time= 7.60414981842041 min 	 loss= 0.6544109796350067
epoch= 2 	 time= 7.676812279224396 min 	 loss= 0.6450033616778013
Finish Training for one subject

Subject num: 36
epoch= 0 	 time= 7.752595802148183 min 	 loss= 0.574573624287606
epoch= 1 	 time= 7.823153611024221 min 	 loss= 0.5550310979723125
epoch= 2 	 time= 7.897586317857106 min 	 loss= 0.5411488681628898
Finish Training for one subject

Subject num: 37
epoch= 0 	 time= 7.969529942671458 min 	 loss= 0.596791502793093
epoch= 1 	 time= 8.044438270727793 min 	 loss= 0.5813326195285127
epoch= 2 	 time= 8.115298287073772 min 	 loss= 0.5684665540182913
Finish Training for one subject

Subject num: 38
epoch= 0 	 ti

In [ ]:
data = torch.load('loss1.pt')

In [ ]:
print(data)

In [ ]:
def show_mask(image, mask):
    """
    Show image and mask
    Args:
        image(numpyarray): The training image
        semantic(numpyarray): The training image segmentation
    """    
    plt.subplot(1,2,1)
    plt.title('image')
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.title('mask')
    plt.imshow(mask)
    plt.show()

In [ ]:
sample=train_data[5]
img = sample['T1a']
mask = sample['parc5a']

show_mask(img.squeeze(), mask)

img = img.unsqueeze(dim = 0)

img = img.to(device)


# feed it to network
scores =  unet(img)
scores = scores.detach().cpu().squeeze().permute(1,2,0)
print(scores.size())
scores = torch.exp(scores)
print(torch.max(scores))
print(torch.min(scores))
a,b = torch.max(scores,dim=2)
plt.imshow(b)
